In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import warnings
import random
from datetime import datetime
random.seed(datetime.now())
warnings.filterwarnings('ignore')

# Make plots larger
plt.rcParams['figure.figsize'] = (10, 6)

In [3]:
# ACCESS_TOKEN = ''

ACCESS_TOKEN = 'EAACEdEose0cBAN4YsIkSihHnSRhZBFx2SPvapIAp73jL85tqOn2Cv7Tq7TgsYBuRQWJyUXtazkHQaOurTxd0luqZC3LZBz41zu7BJmVMPc0qMSyUjDZAj3bPOB0WNLLri5hGJR9jYlkYQBdir7Bt0ynX5hrsUiAArdhqea1Wt9tkgVFBxeebTMub8x5f9oX2mgE19lzQHAZDZD'

In [4]:
import requests # pip install requests
import json

base_url = 'https://graph.facebook.com/me'

# Specify which fields to retrieve
fields = 'id,name,likes'

url = '{0}?fields={1}&access_token={2}'.format(base_url, fields, ACCESS_TOKEN)
print(url)

https://graph.facebook.com/me?fields=id,name,likes&access_token=EAACEdEose0cBAN4YsIkSihHnSRhZBFx2SPvapIAp73jL85tqOn2Cv7Tq7TgsYBuRQWJyUXtazkHQaOurTxd0luqZC3LZBz41zu7BJmVMPc0qMSyUjDZAj3bPOB0WNLLri5hGJR9jYlkYQBdir7Bt0ynX5hrsUiAArdhqea1Wt9tkgVFBxeebTMub8x5f9oX2mgE19lzQHAZDZD


In [5]:
content = requests.get(url).json()
print(json.dumps(content, indent=1))

{
 "id": "876815415686568",
 "name": "Akshay Shinde"
}


In [6]:
import facebook

# Valid API versions are '2.1', '2.2', '2.3', '2.4', '2.5', '2.6', '2.7'

# Create a connection to the Graph API with your access token
g = facebook.GraphAPI(ACCESS_TOKEN, version='2.7')

Ongoing Error Message from Facebook which restricted some of the data  
This week, we are reopening review for developers on Messenger Platform that support a limited number of Pages, allowing them to set up direct Messenger integrations across their own Pages. We will also reenable existing apps using Page conversation APIs to connect to a limited number of Pages.

In [7]:
voice=['ChelseaFC']
feed = g.get_connections(voice[0], 'posts')
feed

{'data': [{'created_time': '2018-04-26T16:07:59+0000',
   'id': '86037497258_1371387389672234',
   'message': "'Can you kick it?' returns with Chelsea Ladies! 🎯",
   'story': 'Chelsea Football Club with Carabao UK.'},
  {'created_time': '2018-04-26T12:16:18+0000',
   'id': '86037497258_10156541326817259',
   'message': "Chelsea Under-18s 5-1 Manchester City!\n\n🏆 Our youngsters were crowned FA Youth Cup champions on this day 1️⃣ year ago!\n\nGet your tickets for tomorrow's first leg at the Bridge!\n🎟: che.lc/tckts"},
  {'created_time': '2018-04-25T17:18:04+0000',
   'id': '86037497258_10156539457232259',
   'message': 'In 7️⃣th heaven! 🔥\n\nThe Blues beat Stoke 7-0 at the Bridge on this day in 2010!'},
  {'created_time': '2018-04-25T15:33:00+0000',
   'id': '86037497258_10156536258177259',
   'message': 'Yokohama is bringing two youth soccer players to London to see a match, tour the field, and meet the Chelsea players themselves! 👌\n\nIf this sounds amazing, then enter today in the Lo

In [8]:
def retrieve_page_feed(page_id, n_posts):
    """Retrieve the first n_posts from a page's feed in reverse
    chronological order."""
    feed = g.get_connections(page_id, 'posts')
    posts = []
    posts.extend(feed['data'])

    while len(posts) < n_posts:
        try:
            feed = requests.get(feed['paging']['next']).json()
            posts.extend(feed['data'])
        except KeyError:
            # When there are no more posts in the feed, break
            print('Reached end of feed.')
            break
            
    if len(posts) > n_posts:
        posts = posts[:n_posts]

    print('{} items retrieved from feed'.format(len(posts)))
    return posts

In [9]:
bs=retrieve_page_feed(voice[0], 33)
bs

33 items retrieved from feed


[{'created_time': '2018-04-26T16:07:59+0000',
  'id': '86037497258_1371387389672234',
  'message': "'Can you kick it?' returns with Chelsea Ladies! 🎯",
  'story': 'Chelsea Football Club with Carabao UK.'},
 {'created_time': '2018-04-26T12:16:18+0000',
  'id': '86037497258_10156541326817259',
  'message': "Chelsea Under-18s 5-1 Manchester City!\n\n🏆 Our youngsters were crowned FA Youth Cup champions on this day 1️⃣ year ago!\n\nGet your tickets for tomorrow's first leg at the Bridge!\n🎟: che.lc/tckts"},
 {'created_time': '2018-04-25T17:18:04+0000',
  'id': '86037497258_10156539457232259',
  'message': 'In 7️⃣th heaven! 🔥\n\nThe Blues beat Stoke 7-0 at the Bridge on this day in 2010!'},
 {'created_time': '2018-04-25T15:33:00+0000',
  'id': '86037497258_10156536258177259',
  'message': 'Yokohama is bringing two youth soccer players to London to see a match, tour the field, and meet the Chelsea players themselves! 👌\n\nIf this sounds amazing, then enter today in the London VIP Soccer Exper

In [10]:
import sqlite3

In [11]:
con = sqlite3.connect('database_final.sqlite')
cursor=con.cursor()

In [12]:

bs_fan = g.get_object(id=voice[0], fields=['fan_count'])
fan=bs_fan['fan_count']
fan

47829983

In [13]:
team_id="8455"
team_id

'8455'

Running the below query will insert data into the database.
But we already have data in the database so it will just duplicate the data.

In [13]:
l=[]
s=[]
c=[]
m=[]
mla=[]
for x in range (20):
    
    try:
        likes = g.get_object(id=bs[x]['id'], 
                         fields=['likes.limit(0).summary(true)'])\
                         ['likes']['summary']['total_count']
    except KeyError:
        likes=0
    try:
        shares = g.get_object(id=bs[x]['id'], 
                         fields=['shares.limit(0).summary(true)'])\
                         ['shares']['count']
    except KeyError:
        shares=0
    try:
        comments = g.get_object(id=bs[x]['id'], 
                         fields=['comments.limit(0).summary(true)'])\
                         ['comments']['summary']['total_count']
    except KeyError:
        comments=0
    try:
        message = g.get_object(id=bs[x]['id'], fields=['message'])['message']
        message.replace('"','')
    except KeyError:
        message="None"
   
    
    format_str = """INSERT INTO Facebook (Team_api_id, Fan_Count, Likes, Shares, Comments, Messages)
                        VALUES("{id}", "{fan}","{likes}","{shares}","{comments}","{message}");"""
    
    sql_command = format_str.format(id=team_id, fan=fan, likes=likes, shares=shares, comments=comments, message=message)
    cursor.execute(sql_command)
        
    con.commit()

In [14]:
pd.read_sql("Select * from facebook;",con)

Team_id  Fan_Count   Likes  Shares  Comments  \
0      10260   73698717    8962      58        65   
1      10260   73698717   29428    1852       833   
2      10260   73698717   16919     604       209   
3      10260   73698717   11025     226        76   
4      10260   73698717    3300      43        16   
5      10260   73698696    8976      58        65   
6      10260   73698696   29447    1854       833   
7      10260   73698696   16928     604       210   
8      10260   73698696   11049     226        76   
9      10260   73698696    3301      43        16   
10     10260      22872       0       0         0   
11     10260      22872       7       0         0   
12     10260      22872       4       0         0   
13     10260      22872       5       0         0   
14     10260      22872       4       0         0   
15     10260      22872       3       0         0   
16     10260      22872       3       0         0   
17     10260      22872       5       0         0   
18     10260      22872       5       0         0   
19     10260      22872       4       0         0   
20     10260      22872       3       0         0   
21     10260      22872       5       0         0   
22     10260  107872907   46629    1685       550   
23     10260  107872907   94248    3748      1233   
24     10260  107872907  172149   16609      2544   
25     10260  107872907  481240  114662     13056   
26     10260  107872907  132990   14662      3429   
27     10260  107872907   69341    8330     19231   
28     10260  107872907   48890    4436       834   
29     10260  107872907  105040   16356      2418   
..       ...        ...     ...     ...       ...   
95      8455   47830931    3559      63       362   
96      8455   47830931   25914     574       201   
97      8455   47830931   95558   10350      1529   
98      8455   47830931   42091    2234       557   
99      8455   47830931   14264     144       123   
100     8455   47830931   10963     113       482   
101     8455   47830931   31288    1622       491   
102     8455   47830931    9441     230       235   
103     8455   47830931    9133     287        99   
104     8455   47830931    9919     331       221   
105     8455   47829979    3752      70        17   
106     8455   47829979   18413    1081       339   
107     8455   47829979   22051    1173       392   
108     8455   47829979      63       2         5   
109     8455   47829979  101005   11970      3830   
110     8455   47829979   12875      80       116   
111     8455   47829979    9600     239        77   
112     8455   47829979   56797    5815      2003   
113     8455   47829979   16365     361       194   
114     8455   47829979   15838     340       163   
115     8455   47829979    8828     446        88   
116     8455   47829979   32928     719       313   
117     8455   47829979    3597      64       362   
118     8455   47829979   26088     575       202   
119     8455   47829979   95781   10356      1534   
120     8455   47829979   42245    2237       559   
121     8455   47829979   14385     145       123   
122     8455   47829979   11066     113       484   
123     8455   47829979   31397    1622       491   
124     8455   47829979    9508     230       235   

                                              Messages  
0                          Four games left to impress!  
1    Cristiano Ronaldo ⚽️⚽️\nWayne Rooney ⚽️⚽️\nDim...  
2    Jesse Lingard’s solo effort against Watford wo...  
3    Awesome from Uber 👏\n\nBringing fans closer to...  
4                                                 None  
5                          Four games left to impress!  
6    Cristiano Ronaldo ⚽️⚽️\nWayne Rooney ⚽️⚽️\nDim...  
7    Jesse Lingard’s solo effort against Watford wo...  
8    Awesome from Uber 👏\n\nBringing fans closer to...  
9                                                 None  
10                  TOP 18 ● MANCHESTER UNITED GOALS ●  
11       Eric Cantona no.1⃣ goal f

### Question 1: What are people saying about me?

We can check what people are saying about somebody by checking the messages the particular teams gets. We have the 'messages' column in our team.

In [15]:
pd.read_sql("select team_id,messages from facebook where team_id = 8455;",con)

Team_id                                           Messages
0      8455  In 7️⃣th heaven! 🔥\n\nThe Blues beat Stoke 7-0...
1      8455  Yokohama is bringing two youth soccer players ...
2      8455                               6️⃣ years ago today!
3      8455  Our Under-19s’ return to the UEFA Youth League...
4      8455  🎥 When you give Callum Hudson-Odoi the camera ...
5      8455  Olivier Giroud brought his love of slalom skii...
6      8455  'We have the chance to get a trophy, an import...
7      8455  'I'm very proud' 👊\n\nThe thoughts of Olivier ...
8      8455         Good luck, Chelsea Ladies Football Club! 🙌
9      8455                      FA Cup final, here we come! 💪
10     8455             Who was your man of the match today? 🤔
11     8455  Our goal scorers today! ⚽️⚽️\n\nWell played Ol...
12     8455                   We are in to the FA Cup final! 🙌
13     8455  MORATAAAA! 🙌\n\nInstant impact from Alvaro who...
14     8455  Sub: Our goalscorer makes way and Alvaro Morat...
15     8455  Antonio Conte makes his first change of the ga...
16     8455  WHAT A START TO THE SECOND HALF! 🔥\n\nThe Blue...
17     8455  The Blues have been on top but at halfway, it'...
18     8455  A man who has plenty of good Wembley memories ...
19     8455  The warm-ups are underway at Wembley! 💪\n\nWha...
20     8455  'Can you kick it?' returns with Chelsea Ladies! 🎯
21     8455  Chelsea Under-18s 5-1 Manchester City!\n\n🏆 Ou...
22     8455  In 7️⃣th heaven! 🔥\n\nThe Blues beat Stoke 7-0...
23     8455  Yokohama is bringing two youth soccer players ...
24     8455                               6️⃣ years ago today!
25     8455  Our Under-19s’ return to the UEFA Youth League...
26     8455  🎥 When you give Callum Hudson-Odoi the camera ...
27     8455  Olivier Giroud brought his love of slalom skii...
28     8455  'We have the chance to get a trophy, an import...
29     8455  'I'm very proud' 👊\n\nThe thoughts of Olivier ...
30     8455         Good luck, Chelsea Ladies Football Club! 🙌
31     8455                      FA Cup final, here we come! 💪
32     8455             Who was your man of the match today? 🤔
33     8455  Our goal scorers today! ⚽️⚽️\n\nWell played Ol...
34     8455                   We are in to the FA Cup final! 🙌
35     8455  MORATAAAA! 🙌\n\nInstant impact from Alvaro who...
36     8455  Sub: Our goalscorer makes way and Alvaro Morat...
37     8455  Antonio Conte makes his first change of the ga...
38     8455  WHAT A START TO THE SECOND HALF! 🔥\n\nThe Blue...
39     8455  The Blues have been on top but at halfway, it'...

### Question 2: How viral are my posts?

We can check how viral someones posts are by summing up the likes of all their posts.

In [16]:
pd.read_sql("SELECT SUM(Likes) FROM facebook WHERE team_id = 8455;",con)

SUM(Likes)
0     1039716

### Question 3: How much influence do my posts have?

We have a field name likes, comments and shares in our database. We can use the sum of these 3 fields to get how much influence my posts have.

In [17]:
pd.read_sql("select team_id, messages,likes+shares+comments as Influence from facebook where messages like '%MORATAAAA%' LIMIT 1;",con)

Team_id                                           Messages  Influence
0     8455  MORATAAAA! 🙌\n\nInstant impact from Alvaro who...      44882

### Question 4: What posts are like mine?

We check the messages which have same words as our message using the 'like' keyword and then display those messages.

In [18]:
pd.read_sql("select messages from facebook where messages like '%Chelsea%';",con)

Messages
0  Yokohama is bringing two youth soccer players ...
1  🎥 When you give Callum Hudson-Odoi the camera ...
2         Good luck, Chelsea Ladies Football Club! 🙌
3  'Can you kick it?' returns with Chelsea Ladies! 🎯
4  Chelsea Under-18s 5-1 Manchester City!\n\n🏆 Ou...
5  Yokohama is bringing two youth soccer players ...
6  🎥 When you give Callum Hudson-Odoi the camera ...
7         Good luck, Chelsea Ladies Football Club! 🙌

### Question 5: What user posts are like mine?

We check the messages which have same words as our message using the 'like' keyword and then display the team who has those messages.

In [19]:
pd.read_sql("select team_id,messages from facebook where messages like '%Chelsea%';",con)

Team_id                                           Messages
0     8455  Yokohama is bringing two youth soccer players ...
1     8455  🎥 When you give Callum Hudson-Odoi the camera ...
2     8455         Good luck, Chelsea Ladies Football Club! 🙌
3     8455  'Can you kick it?' returns with Chelsea Ladies! 🎯
4     8455  Chelsea Under-18s 5-1 Manchester City!\n\n🏆 Ou...
5     8455  Yokohama is bringing two youth soccer players ...
6     8455  🎥 When you give Callum Hudson-Odoi the camera ...
7     8455         Good luck, Chelsea Ladies Football Club! 🙌

### Question 6: Who should i be following?

We have a field named 'fan_count' in the database. We use this field to get the team with max fan count so that we can follow it.

In [20]:
pd.read_sql("select team_id, fan_count from facebook order by fan_count DESC LIMIT 1;",con)

Team_id  Fan_Count
0    10260  107872907

### Question 7: What topics are trending in my domain?

We take the sum of likes, comments and shares from our database to get the trending messages and then display those.

In [21]:
pd.read_sql("select messages, likes+comments+shares as trend from facebook order by trend desc LIMIT 10;",con)

Messages   trend
0  🏁 FT: FC Bayern Munich1-2 Real Madrid C.F.\n(K...  609225
1  🏁 FT: FC Bayern Munich1-2 Real Madrid C.F.\n(K...  608958
2  🇩🇪⚽📺 BEHIND THE SCENES \nFC Bayern Munich - Re...  250027
3  🇩🇪⚽📺 BEHIND THE SCENES \nFC Bayern Munich - Re...  250024
4  🇪🇸🛫🇩🇪 #APorLa13\nWe're on our way to Munich!\n...  228952
5  🇪🇸🛫🇩🇪 #APorLa13\nWe're on our way to Munich!\n...  228951
6  FC Bayern Munich 1-2 Real Madrid C.F.\n⚽ 75' M...  191436
7  FC Bayern Munich 1-2 Real Madrid C.F.\n⚽ 75' M...  191302
8                    🏆 #MondayMotivation\n🔵🔴 #Copa30  162233
9  📝 Real Madrid C.F. XI 🆚 FC Bayern Munich\n#APo...  151114

### Question 8: What keywords/hashtags should i add to my post?

We get the most popular messages by getting the sum of likes, shares and comments. Then check what keywords are used in these messages.

In [22]:
pd.read_sql("Select messages from facebook order by likes+comments+shares desc limit 6;",con)

Messages
0  🏁 FT: FC Bayern Munich1-2 Real Madrid C.F.\n(K...
1  🏁 FT: FC Bayern Munich1-2 Real Madrid C.F.\n(K...
2  🇩🇪⚽📺 BEHIND THE SCENES \nFC Bayern Munich - Re...
3  🇩🇪⚽📺 BEHIND THE SCENES \nFC Bayern Munich - Re...
4  🇪🇸🛫🇩🇪 #APorLa13\nWe're on our way to Munich!\n...
5  🇪🇸🛫🇩🇪 #APorLa13\nWe're on our way to Munich!\n...

### Question 9: Should i follow somebody back?

We take sum of fan_count, likes, shares and comments to see if we should follow somebody back.

In [23]:
pd.read_sql("select team_id, fan_count+likes+shares+comments as follow_score from facebook where follow_score > 108000002 order by follow_score DESC;",con)

Team_id  follow_score
0      8633     108482132
1     10260     108481865
2      8633     108122934
3     10260     108122931
4      8633     108101859
5     10260     108101858
6      8633     108064343
7     10260     108064209
8      8633     108024021
9     10260     108023988
10     8633     108016165
11    10260     108016149

### Question 11: Should i add picture or url to my post?

We check the messages with and without  URL's. We then check the fan_count in both the conditions. Comparing both below we see that it does not matter if we add URL to our posts. (fan_count is same in both the cases)

In [24]:
pd.read_sql("select fan_count,messages from facebook where messages like '%http%' order by fan_count DESC;",con)

Fan_Count                                           Messages
0   107872907  ⚽💪 🏆 #RMUCL \nIt's Champions League night! Fol...
1   107872907  🎟✈🏟 Do you want an unforgettable experience? W...
2   107872907  ⚽💪 🏆 #RMUCL \nIt's Champions League night! Fol...
3   107872907  🎟✈🏟 Do you want an unforgettable experience? W...
4   103190687  👍 Ready to be the next King of Soccer?  ¿Prepa...
5   103190687  A 10-year NHL veteran, with stops at the:\nCan...
6   103190687  Looking forward to the honouring of a football...
7   103190687  🏆 #Copa30\n🎬 PLAYER CAM 🎬\n\n📹 Denis Suárez sh...
8   103190687  🎥 All the best behind-the-scenes bits from #Co...
9    73698717  Jesse Lingard’s solo effort against Watford wo...
10   73698696  Jesse Lingard’s solo effort against Watford wo...
11   47830931  Yokohama is bringing two youth soccer players ...
12   47829979  Yokohama is bringing two youth soccer players ...
13     800014  Killer year for new music in our hometown of S...

In [25]:
pd.read_sql("select fan_count,messages from facebook where messages not like '%http%' order by fan_count DESC;",con)

Fan_Count                                           Messages
0    107872907  ⚽💫🎥 #APorLa132\nRelive our victory in Munich a...
1    107872907  🙌 Marcelo M12\n⚽ GOALS VS:\n🇩🇪 Bayern Munich\n...
2    107872907  FC Bayern Munich 1-2 Real Madrid C.F.\n⚽ 75' M...
3    107872907  🏁 FT: FC Bayern Munich1-2 Real Madrid C.F.\n(K...
4    107872907  📝 Real Madrid C.F. XI 🆚 FC Bayern Munich\n#APo...
5    107872907  🚍➡🏟 #RMUCL 🔴 LIVE\nWatch our arrival at the Al...
6    107872907                                               None
7    107872907  🙌 IT'S MATCHDAY! | ¡DÍA DE PARTIDO! 🙌\n🆚 FC Ba...
8    107872907  🎁🎂🎉 \nHappy Birthday to Raphaël Varane, who tu...
9    107872907  🇩🇪⚽📺 BEHIND THE SCENES \nFC Bayern Munich - Re...
10   107872907  🏟⚽💪 #RMUCL\nOur training session at the Allian...
11   107872907  💫🙌 #APorLa13 \nGet ready for another HUGE #RMU...
12   107872907  🇪🇸🛫🇩🇪 #APorLa13\nWe're on our way to Munich!\n...
13   107872907  🏃‍♂️💨💨 #RMCity \nBring on FC Bayern Munich!\n¡...
14   107872907                      ⏳💫 2 days to go...\n#APorLa13
15   107872907  ⌛⚽🔑 #RMHistory\nRelive Nicolas Anelka's big #R...
16   107872907  😳🚀👏 ¡GOLAZO!\n👟 Iker (Under-13s | Infantil B)\...
17   107872907  ❄😱🙌 SEMI-FINALS\nExcited for the match against...
18   107872907  ⚽💫🎥 #APorLa132\nRelive our victory in Munich a...
19   107872907  🙌 Marcelo M12\n⚽ GOALS VS:\n🇩🇪 Bayern Munich\n...
20   107872907  FC Bayern Munich 1-2 Real Madrid C.F.\n⚽ 75' M...
21   107872907  🏁 FT: FC Bayern Munich1-2 Real Madrid C.F.\n(K...
22   107872907  📝 Real Madrid C.F. XI 🆚 FC Bayern Munich\n#APo...
23   107872907  🚍➡🏟 #RMUCL 🔴 LIVE\nWatch our arrival at the Al...
24   107872907                                               None
25   107872907  🙌 IT'S MATCHDAY! | ¡DÍA DE PARTIDO! 🙌\n🆚 FC Ba...
26   107872907  🎁🎂🎉 \nHappy Birthday to Raphaël Varane, who tu...
27   107872907  🇩🇪⚽📺 BEHIND THE SCENES \nFC Bayern Munich - Re...
28   107872907  🏟⚽💪 #RMUCL\nOur training session at the Allian...
29   107872907  💫🙌 #APorLa13 \nGet ready for another HUGE #RMU...
..         ...                                                ...
81    47829979                               6️⃣ years ago today!
82    47829979  Our Under-19s’ return to the UEFA Youth League...
83    47829979  🎥 When you give Callum Hudson-Odoi the camera ...
84    47829979  Olivier Giroud brought his love of slalom skii...
85    47829979  'We have the chance to get a trophy, an import...
86    47829979  'I'm very proud' 👊\n\nThe thoughts of Olivier ...
87    47829979         Good luck, Chelsea Ladies Football Club! 🙌
88    47829979                      FA Cup final, here we come! 💪
89    47829979             Who was your man of the match today? 🤔
90    47829979  Our goal scorers today! ⚽️⚽️\n\nWell played Ol...
91    47829979                   We are in to the FA Cup final! 🙌
92    47829979  MORATAAAA! 🙌\n\nInstant impact from Alvaro who...
93    47829979  Sub: Our goalscorer makes way and Alvaro Morat...
94    47829979  Antonio Conte makes his first change of the ga...
95    47829979  WHAT A START TO THE SECOND HALF! 🔥\n\nThe Blue...
96    47829979  The Blues have been on top but at halfway, it'...
97      800014  Watch our very own Brian Fennell, aka SYML, li...
98      800014                          Merry Christmas everyone!
99       22872                 TOP 18 ● MANCHESTER UNITED GOALS ●
100      22872      Eric Cantona no.1⃣ goal for Manchester United
101      22872                  Eric Bailly & Zlatan Ibrahimovich
102      22872          Vintage Cristiano Ronaldo against Everton
103      22872                                Fucking Kyle Walker
104      22872  Cristiano Ronaldo surprises a kid on a Madrid'...
105      22872                                               None
106      22872  Bienvenue, Eric! 👑 King Cantona arrived at Old...
107      22872                                               None
108      22872                                               None
109      22872                               

### Question 12: Whats my reach?

The potential reach of an user can be defined as 'fan_count + comments'. We have both these fields in our database.

Reference: https://simplymeasured.com/definition/potential-reach/#sm.0001jco3wmzm7d1swdi209mfljx30 (Whats my potential reach)

In [26]:
pd.read_sql("select team_id, fan_count+comments as Reach from facebook where team_id = 8455 order by Reach DESC LIMIT 1;",con)

Team_id     Reach
0     8455  47834625

## LICENSE

MIT License  

Copyright (c) 2018 Prathamesh Tari and Akshay Shinde

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.